In [3]:
#Download packages
import csv
import pandas as pd
import re
import xml.etree.ElementTree as ET

from tkinter import filedialog
from tkinter import *

In [ ]:
#Create the SDT file that will contain the end product
#Begin by adding the minimum to it


In [65]:
#Display a file explorer and allow user to select a CSV file that contains the desire data
#%%python
def getFile(sdtFile):
    root = Tk()
    root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("text files","*.csv"),("all files","*.*")))
    changePath(root.filename,sdtFile)
    root.destroy()
    return(root.filename)

In [69]:
#Change the path based on the file selected by the user
def changePath(path,sdtFile):
    tree = ET.parse(sdtFile)
    root = tree.getroot()
    
    for elem in root.findall('.//Datasources/Datasource/Host'):
        elem.text=path
    
    for elem in root.findall('.//Datasources/Datasource/Name'):
        elem.text=path
        
    tree.write(sdtFile,encoding="utf-8", xml_declaration=True)

In [5]:
#Read in each column of the CSV file that was selected
def readCSV(csvName):
    df = pd.read_csv(csvName)
    print("reading csv done")
    return df

In [ ]:
#Ask the user how many columns from the file should be used in the visualization
def userInput():
    root = Tk()
    root.withdraw()

    user_inp = simpledialog.askstring(title="SynGlyphX", 
        prompt="How many columns would you like to include in your visualization?")
    user_int = int(user_inp)
    print("Use", user_int, "columns")
    return user_int


In [ ]:
userInput()

In [ ]:
#Ask the user how many children should be used in the visualization
def userInput2():
    root = Tk()
    root.withdraw()

    user_inp = simpledialog.askstring(title="SynGlyphX", 
        prompt="How many children would you like to include in your visualization?")
    user_int = int(user_inp)
    print("Use", user_int, "children")
    return user_int

In [59]:
#Get categorical data
#***********Need to explain the >1 and <6 and color functionality
def getCat(df,colorName):
    x=0
    for name in df.columns:
        if df.dtypes[x] == 'object':
             if len(df[name].astype('category').cat.categories.tolist()) >1 and len(df[name].astype('category').cat.categories.tolist()) <6 and colorName=="": 
                colorName=df.columns[x]
        x= x+1
    return colorName

In [8]:
#Get Categorical Columns
#Return a list of all the categorical columns
def getCatCol(df,colorName):
    return df[colorName].astype('category').cat.categories.tolist()

In [9]:
#Loops through the columns of data
#Adds the numeric columns to a list and passes over categorical object columns
def typeDefiner(df):
    colList = df.columns
    colList
    x = 0
    colNames = []
    for name in colList:
        if df.dtypes[x] == 'object':# or df.dtypes[x] == 'int64':
            pass
        else:
            colNames.append(colList[x])
                
        x = x + 1
    print("objects removed")
    print(colNames)
    return colNames

In [10]:
#
def toNumeric(sdtFile):

    import xml.etree.ElementTree as ET
    #CLEARS COLOR OUT NOW WE CAN INSERT ANYTHING
    file = ET.parse(sdtFile)
    root = file.getroot()
    for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
        elem.clear()

    for parent in root.findall('.//KeyValuePairs/..'):
        for element in parent.findall('KeyValuePairs'):
            parent.remove(element)
   #Try to write here
    for elem in root.findall('.//Color/RGB'):
        check_req_elems = elem.find('.//Function[@type="Text Field To Value"]')
        check_req_elems.type = "Linear Interpolation"
        check_req_elems.set("type","Linear Interpolation")
        attrib = {'type': 'BoundField'}
        minmax = ET.SubElement(check_req_elems, 'MinMax', attrib)
        minmax.text= "\n\t\t\t\t\t\t\t"
        min = ET.SubElement(minmax, 'Min')
        min.text = '0'
        min.tail="\n\t\t\t\t\t\t"
        minmax.tail="\n\t\t\t\t\t"
    file.write(sdtFile,encoding="utf-8", xml_declaration=True)
    print("color fixed")

In [11]:
#
def XMLParser(colNames, sdtFile,colorName):
    print(colNames)

    tree = ET.parse(sdtFile)
    root = tree.getroot()
    count=0;
    print(len(colNames))
    for elem in root.iter('InputField'):
        print(count)
        #count+=1;
        #check_req_elem = elem.find('.//InputField')
        #if check_req_elem.get('name')!="color":
        elem.set('field', colNames[count])
        #else:
         #   elem.set('field', colorName)
        count+=1;
    #tree.write('New DAL 900.sdt')
    tree.write('emptySDT.sdt')
    print("XML updated")

In [67]:
#
def runAll(sdtFile):
    colorCol= None
    colorName=""
    csv = getFile(sdtFile)
    df = readCSV(csv)
    colorName=getCat(df,colorName)
    colorCol=getCatCol(df,colorName)
    colNames = typeDefiner(df)
    colNames.append(colorName)
    XMLParser(colNames, sdtFile,colorName)
    setupColor(colorCol, sdtFile)
   # csv = getFile()
#readCSV(csv)

In [71]:
#%%python
runAll('emptySDT.sdt')

reading csv done
objects removed
['Height', 'Age', 'GPA']
['Height', 'Age', 'GPA', 'Gender']
4
0
1
2
3
XML updated
A
F


### Extra functions

In [4]:
#Search the data for an Elevation column
def elevationFinder(df):
    if df.columns.contains("elevation") or df.columns.contains("Elevation") or df.columns.contains("height"):
        longitude = df["Longitude"]
    else:
        longitude = "none found"
    return longitude

In [3]:
#Search the data for a Latitude column
#Must condiser "LAT" is also found in words such as "later" and "flat"
def latFinder2(df):
    x = 0
    for columns in df.columns:
        searchObj = re.match( r'LAT', df.columns[x], re.I)
        if searchObj:
            #print("found it")
            lat = df.columns[x]
            return lat
        else:
            x = x + 1
            pass

In [2]:
#Search the data for a Longitude column
#Must consider "LONG" is also found in words such as "longer" and "belong"
def longFinder2(df):
    x = 0
    for columns in df.columns:
        searchObj = re.match( r'LONG', df.columns[x], re.I)
        if searchObj:
            #print("found it")
            long = df.columns[x]
            return long
        else:
            x = x + 1
            pass

In [62]:
#
def setupColor(colorCol,sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    x=0
    for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
        for element in elem.findall('KeyValuePair/Key'):
            element.text=colorCol[x]
            print(colorCol[x])
            x=x+1

        if(len(colorCol)>2):
            while x<len(colorCol):
                for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]'):
                    pairs = elem.find('.//KeyValuePairs')
                #fixes spacing so new pair will be at right column/indent area
                    for child in pairs.findall('.//KeyValuePair'):
                        child.tail="\n\t\t\t\t\t\t\t"
                    keyVal =ET.SubElement(pairs, 'KeyValuePair')
                    keyVal.text="\n\t\t\t\t\t\t\t\t"
                    keyVal.tail="\n\t\t\t\t\t\t"
                    keyValP= ET.SubElement(keyVal, 'Key')
                    keyValP.text=colorCol[x]
                    keyValP.tail="\n\t\t\t\t\t\t\t\t"
                    key = ET.SubElement(keyVal, 'Value')
                    if(x==2):
                        key.text = "0,255,0"  
                    elif(x==3):
                        key.text = "255,255,0"
                    elif(x==4):
                        key.text="0,255,255"
                    
                    key.tail="\n\t\t\t\t\t\t\t"
                    x=x+1
       # val = ET.SubElement(keyValP, 'Value')
   # val.text = "90"
   # val.tail"\n"
    file.write('emptySDT1.sdt',encoding="utf-8", xml_declaration=True)


NameError: name 'colorCol' is not defined